In [324]:
import gdown
import pandas as pd
import random
import json

In [263]:
# gdown.download('https://drive.google.com/file/d/17NBXx6l_6znStM52RJOKKKlxnHigavzq/view', fuzzy=True)

In [264]:
# tar -xvzf hackathon_objects.tar.gz

In [2]:
goods_data = pd.read_parquet('../datasets/train_data.parquet')

goods_pairs = pd.read_parquet('../datasets/train_pairs_w_target.parquet')

In [3]:
goods_pairs.head()

,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289
2,1.0,53602615,587809782
3,1.0,53888651,89598677
4,0.0,56930698,551526166


In [4]:
goods_data.head()

,variantid,characteristic_attributes_mapping,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64
0,51195767,"{""85"":[""Партнер-Электро""],""8229"":[""Удлинитель ...","Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[оранжевый],None,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485..."
1,51196903,"{""85"":[""TDM Electric""],""8229"":[""Удлинитель быт...",Колодка TDM Electric пятиместная без заземлени...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],"[[0.42044494, -0.33794826, -0.037247024, 0.165...","[[0.16211876, -0.4455993, 0.6131705, 0.5954206...","[-0.48503304, 0.6264443, 0.6406273, -0.4696772..."
2,52061880,"{""85"":[""MINAMOTO""],""8229"":[""Батарейка""],""5111""...",Батарейка литиевая CR 1/3N 3V (CR11108) Minamo...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарей...",None,"[[0.66729844, -0.023996592, 0.42721167, 0.0205...","[[-0.8638098, -0.1625915, -0.034600798, 0.1337...","[-0.15832633, 0.44736174, 0.31883216, -0.55559..."
3,52067481,"{""85"":[""Duracell""],""8229"":[""Батарейка""],""5111""...","Батарейки DURACELL Ultra Power AA (LR6), 4 шт","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарей...",None,"[[0.3516068, 0.6306597, 0.80962396, -0.0104141...","[[-0.26286322, 0.22858405, 0.4346232, 0.023486...","[-0.42703134, 0.16898727, 0.53922343, -0.53523..."
4,52610752,"{""85"":[""HP""],""8229"":[""Картридж""],""5708"":[""для ...",Картридж лазерный HP 12A Q2612A черный для LJ ...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Расходн...","[черный, чер]","[[-0.009966308, 0.70965785, 0.10790472, -0.411...","[[0.09032486, 0.74205226, 0.30355096, -0.04755...","[-0.4892143, 0.47568643, 0.6505941, -0.4106509..."


In [5]:
goods_data['main_pic_embeddings_resnet_v1'][0][0][0]

0.04603629

In [ ]:
def symm_pairs(df):
    # todo: complete cliques of goods
    
    df2 = df.copy()
    cols = ["variantid1", "variantid2"]
    for c in cols:
        if c not in df2.columns:
            raise UserWarning
    df2 = df2.rename(columns={cols[0]: cols[1], cols[1]: cols[0]})
    return pd.concat([df, df2]).drop_duplicates()

In [ ]:
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader


class ContrastiveDataset(Dataset):
    def __init__(self, original_dataset, pairs_dataset, embedding_columns):
        self.original_dataset = original_dataset.set_index('variantid')
        self.pairs_dataset = pairs_dataset
        self.embedding_columns = embedding_columns
        self.dataset = None
        self.prepare_dataset()
        

    def prepare_dataset(self,):
        neg_pairs =  self.pairs_dataset[ self.pairs_dataset.target ==0.].groupby('variantid1').variantid2.apply(lambda x: list(x)).to_frame()
        pos_pairs =  self.pairs_dataset[ self.pairs_dataset.target ==1.].groupby('variantid1').variantid2.apply(lambda x: list(x)).to_frame()
        self.dataset = pos_pairs.merge(neg_pairs, how = 'outer', left_index = True, right_index = True, indicator = True, suffixes = ['_pos', '_neg'])
        sol = pd.Series(np.where(self.dataset.variantid2_pos.notnull(), self.dataset.variantid2_pos, pd.Series(self.dataset.index).apply(lambda x: [x])))
        self.dataset['variantid2_pos'] = sol.set_axis(self.dataset.index)
        ld = pd.Series(self.dataset.index).sample(frac = 1).apply(lambda x: [x])
        sol = pd.Series(np.where(self.dataset.variantid2_neg.notnull(), self.dataset.variantid2_neg, ld))
        self.dataset['variantid2_neg'] = sol.set_axis(self.dataset.index)
        self.dataset = self.dataset.reset_index()

    def __getitem__(self, index):
        original_idx = self.dataset.loc[index].variantid1
        positive_idx = self.dataset.loc[index].variantid2_pos
        negative_idx = self.dataset.loc[index].variantid2_neg
        if len(positive_idx)>1:
            val = np.random.choice(len(positive_idx), 1)[0]
            positive_idx = positive_idx[val]
        else:
            positive_idx = positive_idx[0]
        if len(negative_idx)>1:
            val = np.random.choice(len(negative_idx), 1)[0]
            negative_idx = negative_idx[val]
        else:
            negative_idx = negative_idx[0]
        original_data = torch.cat([torch.tensor(self.original_dataset.loc[original_idx]['main_pic_embeddings_resnet_v1'][0]), torch.tensor(self.original_dataset.loc[original_idx]['name_bert_64'])], axis = 0)
        positive_pair = torch.cat([torch.tensor(self.original_dataset.loc[positive_idx]['main_pic_embeddings_resnet_v1'][0]), torch.tensor(self.original_dataset.loc[positive_idx]['name_bert_64'])], axis =0)
        negative_pair = torch.cat([torch.tensor(self.original_dataset.loc[negative_idx]['main_pic_embeddings_resnet_v1'][0]), torch.tensor(self.original_dataset.loc[negative_idx]['name_bert_64'])], axis = 0)
        return original_data, positive_pair, negative_pair, original_idx

    def __len__(self):
        return len(self.dataset)-1

    # def get_embeddings(self, index):
    #     embeddings = []
    #     for column in self.embedding_columns:
    #         embedding = self.original_dataset[column].iloc[index]
    #         if embedding.shape[0] == 1:
    #             embedding = embedding[0]
    #         embeddings.append(embedding)
    #     return torch.tensor(np.concatenate(embeddings))

    # def find_positive_pair(self, original_id):
    #     positive_pairs = self.pairs_dataset[self.pairs_dataset["target"] == 1]
    #     matching_pairs = positive_pairs[(positive_pairs["variantid1"] == original_id)]

    #     if len(matching_pairs) > 0:
    #         # Randomly choose a positive pair if multiple pairs are found
    #         chosen_pair = matching_pairs.sample(1)

    #         # Get the ID of the other data point in the pair
    #         pair_id = chosen_pair["variantid2"].values[0]

    #         # Retrieve the corresponding data from the original dataset
    #         positive_row = self.get_embeddings(
    #             self.original_dataset[
    #                 self.original_dataset["variantid"] == pair_id
    #             ].index[0]
    #         )

    #         return positive_row

    #     # Return original data if positive pair is not found
    #     original_row = self.get_embeddings(
    #         self.original_dataset[
    #             self.original_dataset["variantid"] == original_id
    #         ].index[0]
    #     )
    #     return original_row

    # def find_negative_pair(self, original_id):
    #     negative_pairs = self.pairs_dataset[self.pairs_dataset["target"] == 0]
    #     matching_pairs = negative_pairs[(negative_pairs["variantid1"] == original_id)]

    #     if len(matching_pairs) > 0:
    #         # Randomly choose a positive pair if multiple pairs are found
    #         chosen_pair = matching_pairs.sample(1)

    #         # Get the ID of the other data point in the pair
    #         pair_id = chosen_pair["variantid2"].values[0]

    #         # Retrieve the corresponding data from the original dataset
    #         negative_row = self.get_embeddings(
    #             self.original_dataset[
    #                 self.original_dataset["variantid"] == pair_id
    #             ].index[0]
    #         )

    #         return negative_row

    #     # Return random data if positive pair is not found
    #     random_row = self.get_embeddings(random.choice(self.original_dataset.index))
    #     return random_row




# Actual data

In [ ]:
symm_goods_pairs = symm_pairs(goods_pairs)

In [ ]:
pos_pairs = symm_goods_pairs[symm_goods_pairs.target ==1].groupby('variantid1').variantid2.apply(lambda x: list(x)).to_frame()

In [ ]:
#checking if lists contain duplicates
pos_pairs.variantid2.apply(lambda x: len(x) !=len(set(x))).value_counts()

In [ ]:
# добавить все симметричные positive pairs
pos_pairs.variantid2.str.len().value_counts()

In [ ]:
embedding_columns = ['main_pic_embeddings_resnet_v1', 'name_bert_64']

contrastive_dataset = ContrastiveDataset(goods_data, symm_goods_pairs, embedding_columns)

In [ ]:
contrastive_dataloader = DataLoader(contrastive_dataset, batch_size=128, shuffle=False, drop_last= True)
next(iter(contrastive_dataloader))[0].shape

In [ ]:
yt[0].shape, yt[1].shape, yt[2].shape

# Basic train example

In [ ]:
contrastive_dataset.dataset.shape

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

In [252]:
#!g1.1

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#!g1.1
class ContrastiveModel(nn.Module):
    def __init__(self, input_size, hidden_size, embedding_size):
        super(ContrastiveModel, self).__init__()
        
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, embedding_size)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def triplet_loss(anchor, positive, negative, margin=1.0):
    distance_positive = torch.norm(anchor - positive, dim=1)
    distance_negative = torch.norm(anchor - negative, dim=1)
    loss = torch.clamp(distance_positive - distance_negative + margin, min=0.0).mean()
    return loss

# Example parameters
input_size = 192
hidden_size = 64
embedding_size = 32
learning_rate = 0.001
num_epochs = 10

# Create an instance of the model
model = ContrastiveModel(input_size, hidden_size, embedding_size).to(device)

# Define the optimizer and the loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train loop
for epoch in tqdm(range(num_epochs)):
    total_loss = 0.0
    
    for original_data, positive_pair, negative_pair, _ in tqdm(contrastive_dataloader):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        anchor = model(original_data.to(device))
        positive = model(positive_pair.to(device))
        negative = model(negative_pair.to(device))
        
        # Compute the triplet loss
        loss = triplet_loss(anchor, positive, negative)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
   #     print(loss.item())
        total_loss += loss.item()
    
    average_loss = total_loss / len(contrastive_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}")


In [ ]:
torch.save(model.state_dict(), 'contrastive_ozon.pt')

In [ ]:
def predict(model, dataloader):
    y_idxs = []
    y_embed = []
    for orig, _, _, idx in tqdm(dataloader):
        embed = model(orig.to(device))
        y_idxs.extend([i.detach().cpu().item() for i in idx])
        y_embed.extend([[j.item() for j in i.detach().cpu()] for i in embed])
    return y_idxs, y_embed

y_idxs, y_embed = predict(model, contrastive_dataloader)

        


In [ ]:
embed = pd.DataFrame({'variant': y_idxs, 'embed': y_embed})

In [ ]:
classif = goods_pairs.merge(embed, left_on = 'variantid1', right_on = 'variant', how = 'left', suffixes = ['', '_1'])

In [ ]:
classif = classif.merge(embed, left_on = 'variantid2', right_on='variant', how = 'left', suffixes = ['', '_2'])

In [ ]:
classif.drop(['variant', 'variant_2'],axis = 1, inplace = True)

In [ ]:
classif

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_pic_features(main_pic_embeddings_1,
                     main_pic_embeddings_2,
                     percentiles):
    """Calculate distances percentiles for 
    pairwise pic distances. Percentiles are useful 
    when product has several pictures.
    """
    
    if main_pic_embeddings_1 is not None and main_pic_embeddings_2 is not None:     
        dist_m = cosine_similarity(
            np.array(main_pic_embeddings_1)[None, ...], np.array(main_pic_embeddings_2)[None, ...]
        )
    else:
        dist_m = np.array([[-1]])

    pair_features = []
    pair_features += np.percentile(dist_m, percentiles).tolist()

    return pair_features

In [ ]:
from functools import partial
classif = classif.dropna()
get_pic_features_func = partial(
    get_pic_features,
    percentiles=[0, 25, 50]
)
classif[["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc"]] = (
    classif[["embed", "embed_2"]].apply(
        lambda x: pd.Series(get_pic_features_func(*x)), axis=1
    )
)

In [ ]:
classif

In [ ]:
classif = classif.merge(goods_data[['variantid', 'categories']], left_on = 'variantid1', right_on = 'variantid', how = 'left')

In [ ]:
classif["cat3"] = classif["categories"].apply(lambda x: json.loads(x)["3"])

In [ ]:
cat3_counts = classif["cat3"].value_counts().to_dict()

In [ ]:
classif["cat3_grouped"] = classif["cat3"].apply(lambda x: x if cat3_counts[x] > 1000 else "rest")

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(
    classif[['target', "variantid1", "variantid2", 'pic_dist_0_perc', 'cat3_grouped']], 
    test_size=0.1, random_state=42, stratify=classif[["target", "cat3_grouped"]]
)

y_test = X_test[["target", "variantid1", "variantid2" ]]
X_test = X_test.drop(["target"], axis=1)

y_train = X_train["target"]

X_train = X_train.drop(["target", "variantid1", "variantid2"], axis=1)

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
model = CatBoostClassifier()

model.fit(
    X_train[['pic_dist_0_perc']], y_train,
    plot=True,
    verbose=True,
    use_best_model=True,
    early_stopping_rounds=50,
    metric_period=100
)

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc


def pr_auc_macro(
    target_df: pd.DataFrame,
    predictions_df: pd.DataFrame,
    prec_level: float = 0.75,
    cat_column: str = "cat3_grouped"
) -> float:
    
    df = target_df.merge(predictions_df, on=["variantid1", "variantid2"])
    
    y_true = df["target"]
    y_pred = df["scores"]
    categories = df[cat_column]
    
    weights = []
    pr_aucs = []

    unique_cats, counts = np.unique(categories, return_counts=True)

    for i, category in enumerate(unique_cats):
        cat_idx = np.where(categories == category)[0]
        y_pred_cat = y_pred[cat_idx]
        y_true_cat = y_true[cat_idx]

        y, x, thr = precision_recall_curve(y_true_cat, y_pred_cat)
        gt_prec_level_idx = np.where(y >= prec_level)[0]

        try:
            pr_auc_prec_level = auc(x[gt_prec_level_idx], y[gt_prec_level_idx])
            if not np.isnan(pr_auc_prec_level):
                pr_aucs.append(pr_auc_prec_level)
                weights.append(counts[i] / len(categories))
        except ValueError as err:
            pr_aucs.append(0)
            weights.append(0)
    return np.average(pr_aucs, weights=weights)

In [ ]:
X_test["scores"] = model.predict_proba(X_test[['pic_dist_0_perc']])[:, 1]

In [ ]:
pr_auc_macro_metr = pr_auc_macro(
    target_df=y_test, 
    predictions_df=X_test,
    prec_level=0.75,
    cat_column="cat3_grouped"
)

pr_auc_macro_metr

In [ ]:
#используется только одна картинка, можно делать positive pairs со всеми

In [ ]:
X_test["scores"].hist()